<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_04/TS%20-%20W4%20-%2004%20-%20LSTM%20(Laborat%C3%B3rio%201).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Uso de convoluções com LSTMs

Neste laboratório, você usará como base os modelos RNN criados em notebooks anteriores e acrescentará uma camada de convolução a eles.

Como você viu em cursos anteriores, os filtros de convolução também podem capturar recursos de sequências, portanto, é bom experimentá-los ao explorar as arquiteturas de modelos. Vamos começar!

## Importações

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Utilitários

Você plotará o MAE e a perda posteriormente, portanto, o `plot_series()` foi estendido para ter mais funcionalidade de rotulagem. Os utilitários para gerar os dados sintéticos são os mesmos dos laboratórios anteriores.

In [ ]:
def plot_series(x, y, format="-", start=0, end=None,
                title=None, xlabel=None, ylabel=None, legend=None ):
    """
    Visualiza dados de séries temporais

    Args:
      x (matriz de int) - contém valores para o eixo x
      y (matriz de int ou tupla de matrizes) - contém os valores para o eixo y
      format (string) - estilo de linha ao plotar o gráfico
      start (int) - primeiro passo de tempo a ser plotado
      end (int) - último passo de tempo a ser plotado
      title (string) - título do gráfico
      xlabel (string) - rótulo do eixo x
      ylabel (string) - rótulo do eixo y
      legend (lista de strings) - legenda para o gráfico
    """

    # Configurar as dimensões da figura do gráfico
    plt.figure(figsize=(10, 6))

    # Verificar se há mais de duas séries para plotar
    if type(y) is tuple:

      # Fazer um loop sobre os elementos y
      for y_curr in y:

        # Plote os valores x e y atuais
        plt.plot(x[start:end], y_curr[start:end], format)

    else:
      # Plote os valores x e y
      plt.plot(x[start:end], y[start:end], format)

    # Rotular o eixo x
    plt.xlabel(xlabel)

    # Rotular o eixo y
    plt.ylabel(ylabel)

    # Configurar a legenda
    if legend:
      plt.legend(legend)

    # Definir o título
    plt.title(title)

    # Sobrepor uma grade no gráfico
    plt.grid(True)

    # Desenhe o gráfico na tela
    plt.show()

def trend(time, slope=0):
    """
    Gera dados sintéticos que seguem uma linha reta com um valor de inclinação.

    Args:
      time (vetor de int) - contém as etapas de tempo
      slope (float) - determina a direção e a inclinação da linha

    Retorna:
      series (matriz de float) - medições que seguem uma linha reta
    """

    # Calcula a série linear dada a inclinação
    series = slope * time

    return series

def seasonal_pattern(season_time):
    """
    Apenas um padrão arbitrário, você pode alterá-lo se desejar

    Args:
      season_time (matriz de float) - contém as medições por etapa de tempo

    Retorna:
      data_pattern (matriz de float) - contém os valores de medição revisados de acordo com o padrão definido.
                                  de acordo com o padrão definido
    """

    # Gerar os valores usando um padrão arbitrário
    data_pattern = np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

    return data_pattern

def seasonality(time, period, amplitude=1, phase=0):
    """
    Repete o mesmo padrão em cada período

    Args:
      time (matriz de int) - contém as etapas de tempo
      period (int) - número de etapas de tempo antes da repetição do padrão
      amplitude (int) - valor de pico medido em um período
      phase (int) - número de etapas de tempo para deslocar os valores medidos

    Retorna:
      data_pattern (matriz de float) - dados sazonais dimensionados pela amplitude definida
    """

    # Definir os valores medidos por período
    season_time = ((time + phase) % period) / period

    # Gera os dados sazonais dimensionados pela amplitude definida
    data_pattern = amplitude * seasonal_pattern(season_time)

    return data_pattern

def noise(time, noise_level=1, seed=None):
    """Gera um sinal ruidoso normalmente distribuído

    Args:
      time (matriz de int) - contém as etapas de tempo
      noise_level (float) - fator de escala para o sinal gerado
      seed (int) - semente do gerador de números para repetibilidade

    Retorna:
      noise (matriz de float) - o sinal ruidoso
    """

    # Inicializar o gerador de números aleatórios
    rnd = np.random.RandomState(seed)

    # Gerar um número aleatório para cada etapa de tempo e dimensionar pelo nível de ruído
    noise = rnd.randn(len(time)) * noise_level

    return noise

## Gerar os dados sintéticos

In [ ]:
# Hyperparâmetros
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Criar a série
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)

# Atualizar com ruído
series += noise(time, noise_level, seed=42)

# Plotar os resultados
plot_series(time, series, xlabel='Time', ylabel='Value')

## Dividir o conjunto de dados

In [ ]:
# Definir o tempo de divisão
split_time = 1000

# Obter o conjunto de treino
time_train = time[:split_time]
x_train = series[:split_time]

# Obter o conjunto de validação
time_valid = time[split_time:]
x_valid = series[split_time:]

## Preparar recursos e rótulos

Conforme mencionado nas palestras, você pode experimentar diferentes tamanhos de lote aqui e ver como isso afeta seus resultados.

In [ ]:
# Hyperparâmetros
window_size = 20
batch_size = 16
shuffle_buffer_size = 1000

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Gera janelas de conjunto de dados

    Args:
      series (array of float) - contém os valores da série temporal
      window_size (int) - o número de etapas de tempo para calcular a média
      batch_size (int) - o tamanho do lote
      shuffle_buffer(int) - tamanho do buffer a ser usado para o método shuffle

    Retorna:
      dataset (TF Dataset) - Conjunto de dados TF contendo janelas de tempo
    """

    # Gerar um conjunto de dados TF a partir dos valores da série
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Janela de dados, mas só pega aqueles com o tamanho especificado
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)

    # Achatar as janelas, colocando seus elementos em um único lote
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Criar tuplas com recursos e rótulos
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Embaralhar as janelas
    dataset = dataset.shuffle(shuffle_buffer)

    # Criar lotes de janelas
    dataset = dataset.batch(batch_size).prefetch(1)

    return dataset

In [ ]:
# Gerar as janelas do conjunto de dados
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

## Construir o modelo

Aqui está a arquitetura do modelo que você usará. Ela é muito semelhante à última RNN que você construiu, mas com a camada [Conv1D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D) na entrada. Um [argumento](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D#args) importante aqui é o `padding`. Para dados de série temporal, é uma boa prática não permitir que os cálculos de uma determinada etapa de tempo sejam afetados por valores futuros. Por exemplo:
* Digamos que você tenha uma pequena janela de série temporal com os seguintes valores: `[1, 2, 3, 4, 5]`. Isso significa que o valor `1` está em `t=0`, `2` está em `t=1`, etc.
* Se você tiver um kernel 1D de tamanho `3`, a primeira convolução será para os valores em `[1, 2, 3]`, que são valores de `t=0` a `t=2`.
* Quando você passa isso para o primeiro intervalo de tempo do `LSTM` após a convolução, significa que o valor em `t=0` do LSTM depende de `t=1` e `t=2`, que são valores futuros.
* Para dados de série temporal, você deseja que os cálculos dependam apenas das etapas de tempo atuais e anteriores.
* Uma maneira de fazer isso é preencher a matriz dependendo do tamanho do kernel e do stride. Para um tamanho de kernel de 3 e stride de 1, a janela pode ser preenchida da seguinte forma: `[0, 0, 1, 2, 3, 4, 5]`. O `1` ainda está em `t=0` e dois zeros são anexados para simular valores no passado.
* Dessa forma, a primeira cadeia estará em `[0, 0, 1]` e não conterá nenhum valor futuro quando for passada para as camadas subsequentes.

A camada `Conv1D` faz esse tipo de preenchimento definindo `padding=causal` e você verá isso abaixo.

In [ ]:
# Redefinir estados gerados pelo Keras
tf.keras.backend.clear_session()

# COnstruir o modelo
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[window_size, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])

# Imprimir sumário do modelo
model.summary()

## Ajuste a taxa de aprendizado

Nos laboratórios anteriores, você usou modelos diferentes para ajuste e treinamento. Essa é uma abordagem válida, mas você também pode usar o mesmo modelo para ambos. Antes de fazer o ajuste, você pode usar o método [`get_weights()`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#get_weights) para poder redefini-lo posteriormente.

In [ ]:
# Obter pesos iniciais
init_weights = model.get_weights()

After that, you can tune the model as usual.

In [ ]:
# Definir o agendador de taxa de aprendizado
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Inicializar o otimizador
optimizer = tf.keras.optimizers.SGD(momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# Treinar o modelo
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
# Definir a matriz de taxa de aprendizado
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Definir o tamanho da figura
plt.figure(figsize=(10, 6))

# Definir a grade
plt.grid(True)

# Plotar a perda em escala logarítmica
plt.semilogx(lrs, history.history["loss"])

# Aumentar o tamanho dos tickmarks
plt.tick_params('both', length=10, width=1, which='both')

# Definir os limites do gráfico
plt.axis([1e-8, 1e-3, 0, 50])

## Treinar o modelo

Para redefinir os pesos, basta chamar a função [`set_weights()`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#set_weights) e passar os pesos salvos anteriormente.

In [ ]:
# Redefinir estados gerados pelo Keras
tf.keras.backend.clear_session()

# Redefinir os pesos
model.set_weights(init_weights)

Em seguida, você pode definir os parâmetros de treinamento e iniciar o treinamento.

In [ ]:
# Definir a taxa de aprendizado
learning_rate = 1e-7

# Definir o otimizador
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

In [ ]:
# Treinar o modelo
history = model.fit(train_set,epochs=500)

O treinamento pode ser um pouco instável, especialmente quando os pesos começam a convergir, portanto, convém visualizá-lo para ver se ainda há tendência de queda. As épocas anteriores podem dominar o gráfico, portanto, também é bom dar zoom nas partes posteriores do treinamento para observar adequadamente os parâmetros. O código abaixo visualiza o `mae` e a `perda` para todas as épocas e também aumenta o zoom nos últimos 80%.

In [ ]:
# Obtenha a MAE e a perda do registro histórico
mae=history.history['mae']
loss=history.history['loss']

# Obter o número de épocas
epochs=range(len(loss))

# Plotar mae e loss
plot_series(
    x=epochs,
    y=(mae, loss),
    title='MAE e Loss',
    xlabel='Épocas',
    legend=['MAE', 'Loss']
    )

# Plote apenas os últimos 80% das épocas
zoom_split = int(epochs[-1] * 0.2)
epochs_zoom = epochs[zoom_split:]
mae_zoom = mae[zoom_split:]
loss_zoom = loss[zoom_split:]

# # Plotar perdas e ganhos com zoom
plot_series(
    x=epochs_zoom,
    y=(mae_zoom, loss_zoom),
    title='MAE e Loss',
    xlabel='Épocas',
    legend=['MAE', 'Loss']
    )

## Previsão do modelo

Quando o treinamento estiver concluído, você poderá gerar as previsões do modelo e plotá-las em relação ao conjunto de validação.

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    """Usa um modelo de entrada para gerar previsões em janelas de dados

    Args:
      model (TF Keras Model) - modelo que aceita janelas de dados
      series (matriz de float) - contém os valores da série temporal
      window_size (int) - o número de etapas de tempo a serem incluídas na janela
      batch_size (int) - o tamanho do lote

    Retorna:
      forecast (matriz numpy) - matriz que contém as previsões
    """

    # Gerar um conjunto de dados TF a partir dos valores da série
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Janela de dados, mas só pega aqueles com o tamanho especificado
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)

    # Achatar as janelas, colocando seus elementos em um único lote
    dataset = dataset.flat_map(lambda w: w.batch(window_size))

    # Criar lotes de janelas
    dataset = dataset.batch(batch_size).prefetch(1)

    # Obter previsões sobre todo o banco de dados
    forecast = model.predict(dataset)

    return forecast

In [ ]:
# Reduzir a série original
forecast_series = series[split_time-window_size:-1]

# Use a função auxiliar para gerar previsões
forecast = model_forecast(model, forecast_series, window_size, batch_size)

# Solte os eixos unidimensionais
results = forecast.squeeze()

# Plotar os resultados
plot_series(time_valid, (x_valid, results))

Em seguida, você pode calcular as métricas como de costume.

In [ ]:
## Calcular o MAE e o MSE
print(tf.keras.metrics.mean_squared_error(x_valid, results).numpy())
print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())

## Adição de um _callback_ para interrupção antecipada

Nesta seção opcional, você adicionará um retorno de chamada para interromper o treinamento quando uma métrica for atingida. Você já fez isso no antes e agora seria um bom momento para revisar.

Primeiro, você precisa preparar um conjunto de validação que o modelo possa usar e monitorar. Conforme mostrado no laboratório anterior, você pode usar a função `windowed_dataset()` para preparar esse conjunto.

In [ ]:
# Gerar janelas de dados a partir do conjunto de validação
val_set = windowed_dataset(x_valid, window_size, batch_size, shuffle_buffer_size)

Você pode redefinir os pesos do modelo ou simplesmente continuar de onde parou.

In [ ]:
# Descomente se você quiser redefinir os peso
# model.set_weights(init_weights)

Em seguida, você definirá uma função de retorno de chamada que será executada a cada final de época. Dentro dela, você definirá a condição para interromper o treinamento. Para este laboratório, você definirá a interrupção quando o `val_mae` for menor que `5,2`.

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Interrompe o treinamento quando uma determinada métrica é atingida

    Args:
      epoch (integer) - índice de época (obrigatório, mas não utilizado na definição de função abaixo)
      logs (dict) - resultados da métrica da época de treinamento
    '''

    # Verificar o conjunto de validação MAE
    if(logs.get('val_mae') < 5.2):

      # Parar se o limite for atingido
      print("\nO valor MAE necessário foi atingido, portanto, o treinamento está sendo encerrado!")
      self.model.stop_training = True

# Instanciar a classe
callbacks = myCallback()

Lembre-se de definir uma taxa de aprendizado apropriada aqui. Se você estiver começando com pesos aleatórios, talvez queira usar a mesma taxa usada anteriormente. No entanto, se você não redefiniu os pesos, poderá usar uma taxa de aprendizagem mais baixa para que o modelo possa aprender melhor. Se tudo correr bem, o treinamento será interrompido antes que o conjunto de 500 épocas seja concluído.

In [ ]:
# Definir a taxa de aprendizado
learning_rate = 4e-8

# Definir o otimizador
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# Treinar o modelo
history = model.fit(train_set, epochs=500, validation_data=val_set, callbacks=[callbacks])

Na prática, você normalmente tem um conjunto de testes separado para avaliar em relação a dados não vistos. No entanto, para este exercício, o conjunto de dados já é muito pequeno, portanto, vamos usar o mesmo conjunto de validação apenas para verificar se os resultados são comparáveis aos que você obteve anteriormente.

In [ ]:
# Reduzir a série original
forecast_series = series[split_time-window_size:-1]

# Use a função auxiliar para gerar previsões
forecast = model_forecast(model, forecast_series, window_size, batch_size)

# Descartar eixo unidimensional
results = forecast.squeeze()

# Plotrar os resultados
plot_series(time_valid, (x_valid, results))

As métricas computadas aqui serão ligeiramente diferentes das mostradas na saída de treinamento porque há mais pontos para avaliar. Lembre-se de que `x_valid` tem 461 pontos que correspondem a `t=1000` a `t=1460`. O `val_set` (que é um conjunto de dados janelados do `x_valid`), por outro lado, tem apenas 441 pontos porque não pode gerar previsões para `t=1000` a `t=1019` (ou seja, o janelamento começará aí).

In [ ]:
## Calcular o MAE e o MSE
print(tf.keras.metrics.mean_squared_error(x_valid, results).numpy())
print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())

## Resumo

Neste laboratório, você conseguiu criar e treinar um modelo CNN-RNN para previsão. Isso conclui a série de notebooks sobre treinamento com dados sintéticos.

Nos próximos laboratórios, você analisará um conjunto de dados de séries temporais do mundo real, especialmente ciclos de manchas solares!